# 抽奖名单

In [107]:
import pymssql
import time
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")  # 忽略UserWarning兼容性警告

In [108]:
file_fz = pd.read_excel('C:/Users/15516/Desktop/全国企查查数据0704_分列.xlsx')

server = "192.168.1.250"
# server = "120.253.186.206:12513"
user = "baseline2"
password = "qwert123!@#"
database = "Lichi"  

# 建立数据库连接
conn = pymssql.connect(server=server, user=user, password=password, database=database)

In [109]:
dictread_sh = {
    'area' : '上海',
    'starttime' : '2023-07-01 00:00:00',
    'endtime'   : '2023-08-01 00:00:00',
    'isum': 599,
    'brand': '马尼'
}

dictread_bj = {
    'area' : '北京',
    'starttime' : '2023-07-01 00:00:00',
    'endtime'   : '2023-08-01 00:00:00',
    'isum': 599,
    'brand': '马尼'
}

dictread_gz = {
    'area' : '广州',
    'starttime' : '2023-07-01 00:00:00',
    'endtime'   : '2023-08-01 00:00:00',
    'isum': 599,
    'brand': '马尼'
}

In [110]:
def query(dictread):
    
    area = dictread['area']
    starttime = dictread['starttime']
    endtime = dictread['endtime']
    isum = dictread['isum']
    brand = dictread['brand']
    
    
    area_mapping = {
        '上海': 'lichi',
        '北京': 'lichi_beijing',
        '广州': 'lichi_guangzhou'
    }
    lichi = area_mapping.get(area, '')
    
    
    

    query_1 = f'''
    SELECT 
        CASE
            WHEN cSOCode LIKE '%[_][0-9]' THEN SUBSTRING(cSOCode, 1, LEN(cSOCode) - 2)
            ELSE cSOCode
        END AS '订单',
        cInvoiceTitle AS '客户',
        cDeliveryLinkTel AS '手机号',
        cDeliveryAddressee AS '收货人',
        SUM(isum) AS '发货金额'
    FROM
        {lichi}..SA_SalesOrderMaster ma
    INNER JOIN
        {lichi}..SA_SalesOrderDetail de ON ma.somid = de.somid
    WHERE
        cInvoiceTitle NOT LIKE '%上海励齿%'
        AND cInvoiceTitle NOT LIKE '%北京励齿%'
        AND cInvoiceTitle NOT LIKE '%广州励齿%'
        and cInvoiceTitle NOT LIKE '%贸易%'
        AND Status != '草稿'
        AND (SELECT cbrandname FROM {lichi}..inventory i WHERE de.cinvcode = i.cinvcode) LIKE '%{brand}%'
        AND dInputDate BETWEEN '{starttime}' AND '{endtime}'
        AND cSettlement !='06'
        
    GROUP BY
        CASE
            WHEN cSOCode LIKE '%[_][0-9]' THEN SUBSTRING(cSOCode, 1, LEN(cSOCode) - 2)
            ELSE cSOCode
        END,
        cInvoiceTitle,
        cDeliveryLinkTel,
        cDeliveryAddressee
    HAVING
        SUM(isum) > {isum};
    '''    
    
    query_2 = f'''
    SELECT 
    CASE
        WHEN cSOCode LIKE '%[_][0-9]' THEN SUBSTRING(cSOCode, 1, LEN(cSOCode) - 2)
        ELSE cSOCode
    END AS '订单',
    cInvoiceTitle AS '客户',
    cDeliveryLinkTel AS '手机号',
    cDeliveryAddressee AS '收货人',
    SUM(isum) AS '发货金额',
    CASE
        WHEN cInvoiceTitle LIKE '%诊所%' OR
             cInvoiceTitle LIKE '%门诊%' OR
             cInvoiceTitle LIKE '%卫生所%' OR
             cInvoiceTitle LIKE '%卫生室%' OR
             cInvoiceTitle LIKE '%口腔%' OR
             cInvoiceTitle LIKE '%牙齿%' OR
             cInvoiceTitle LIKE '%医院%' OR
             cInvoiceTitle LIKE '%牙管家%' THEN 'zd'
        WHEN cInvoiceTitle LIKE '%科技%' OR
             cInvoiceTitle LIKE '%器械%' OR
             cInvoiceTitle LIKE '%商贸%' OR
             cInvoiceTitle LIKE '%网络%' OR
             cInvoiceTitle LIKE '%科贸%' OR
             cInvoiceTitle LIKE '%贸易%' OR
             cInvoiceTitle LIKE '%实业%' OR
             cInvoiceTitle LIKE '%日用%' OR
             cInvoiceTitle LIKE '%技术%' OR
             cInvoiceTitle LIKE '%美容%' OR
             cInvoiceTitle LIKE '%美体%' OR
             cInvoiceTitle LIKE '%制作%' OR
             cInvoiceTitle LIKE '%管理%' OR
             cInvoiceTitle LIKE '%材料%' OR
             cInvoiceTitle LIKE '%器材%' OR
             cInvoiceTitle LIKE '%经贸%' OR
             cInvoiceTitle LIKE '%设备%' OR
             cInvoiceTitle LIKE '%健康咨询%' OR
             cInvoiceTitle LIKE '%加工%' THEN 'jx'
        ELSE 'zd'
    END AS '客户类别'
    FROM
        {lichi}..SA_SalesOrderMaster ma
    INNER JOIN
        {lichi}..SA_SalesOrderDetail de ON ma.somid = de.somid
    WHERE
        cInvoiceTitle NOT LIKE '%上海励齿%'
        AND cInvoiceTitle NOT LIKE '%北京励齿%'
        AND cInvoiceTitle NOT LIKE '%广州励齿%'
        AND Status != '草稿'
        AND (SELECT cbrandname FROM {lichi}..inventory i WHERE de.cinvcode = i.cinvcode) LIKE '%{brand}%'
        AND dInputDate BETWEEN '{starttime}' AND '{endtime}'
        AND cSettlement !='06'
    GROUP BY
        CASE
            WHEN cSOCode LIKE '%[_][0-9]' THEN SUBSTRING(cSOCode, 1, LEN(cSOCode) - 2)
            ELSE cSOCode
        END,
        cInvoiceTitle,
        cDeliveryLinkTel,
        cDeliveryAddressee
    HAVING
        SUM(isum) > {isum};
    ''' 
    
    if area == '北京':
        return query_2
    else:
        return query_1

 
    
       
    
def test(dictread):
    area = dictread['area']
    starttime = dictread['starttime']
    endtime = dictread['endtime']
    isum = dictread['isum']
    
    query_s = query(dictread)
    
    file = pd.read_sql(query_s, conn)
    
    if area == '北京':
        file =  deal_with_bj(area,file,isum)
    else:
        file =  deal_with(area,file,isum)
        
    return file


def deal_with_bj(area,file,isum):
    # 保留客户类别为'zd'的行
    file_bj = file[file["客户类别"] == "zd"]

    # 将file_bj和file_fz按照客户名称合并，使用左连接方式
    file_bj = file_bj.merge(file_fz, left_on='客户', right_on='企业名称', how='left')

    # 删除客户类型为'经销商'的行
    file_bj = file_bj[file_bj['客户类型'] != '经销商']

    # 添加地区列并设置为'北京'
    file_bj['地区'] = area

    # 计算抽奖次数并转换为整数类型
    file_bj['抽奖次数'] = (file_bj['发货金额'] // isum).astype(int)

    # 根据抽奖次数生成次数列表
    file_bj['次数'] = file_bj['抽奖次数'].apply(lambda x: list(range(1, x+1)))

    # 将次数列表展开为多行
    file_bj = file_bj.explode('次数').reset_index(drop=True)

    # 保留指定列
    file_bj = file_bj[['客户', '收货人', '手机号', '订单', '发货金额', '次数','地区']]

    return file_bj

def deal_with(area,file,isum):
    
    # 将file和file_fz按照客户名称合并，使用左连接方式
    file = file.merge(file_fz, left_on='客户', right_on='企业名称', how='left')

    # 删除客户类型为'经销商'的行
    file = file[file['客户类型'] != '经销商']

    # 添加地区列并设置为area
    file['地区'] = area

    # 计算抽奖次数并转换为整数类型
    file['抽奖次数'] = (file['发货金额'] // isum).astype(int)

    # 根据抽奖次数生成次数列表
    file['次数'] = file['抽奖次数'].apply(lambda x: list(range(1, x+1)))

    # 将次数列表展开为多行
    file = file.explode('次数').reset_index(drop=True)

    # 保留指定列
    file = file[['客户', '收货人', '手机号', '订单', '发货金额', '次数','地区']]

    
    return file


def encrypt_name(name):
    if len(name) <= 2:
        return name[0] + '*'*(len(name)-1)
    else:
        return name[0] + '*'*(len(name)-2) + name[-1]
    
    
def encrypt_phone(phone):
    if len(phone) <= 7:
        return phone[:3] + '*'*(len(phone)-7) + phone[-4:]
    else:
        return phone[:3] + '*'*(len(phone)-7) + phone[-4:]
    
    
def split_df_into_columns(df, chunk_size=100):
    # 计算拆分后的列数
    num_columns = (len(df) + chunk_size - 1) // chunk_size

    # 创建一个新的DataFrame，每100个数据为一列
    new_df = pd.DataFrame()
    for i in range(num_columns):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(df))
        new_col = df.iloc[start_idx:end_idx].reset_index(drop=True)
        new_col_name = f'Column_{i + 1}'
        new_df[new_col_name] = new_col

    return new_df

In [111]:
file_sh = test(dictread_sh)
file_bj = test(dictread_bj)
file_gz = test(dictread_gz)

In [112]:
# 合并三个DataFrame
merged_df = pd.concat([file_sh, file_bj, file_gz], ignore_index=True)

# 重新设置序号
merged_df.reset_index(drop=True, inplace=True)

# 对收货人列应用加密函数
merged_df['收货人'] = merged_df['收货人'].apply(encrypt_name)

# 对手机号列应用加密函数
merged_df['手机号'] = merged_df['手机号'].apply(encrypt_phone)

# 创建新列"finall"，用于存储组合后的数据
merged_df['finall'] = merged_df['客户'] + '-' + merged_df['收货人'] + '-' + merged_df['手机号'] + '-' + merged_df['订单'] + '-' + merged_df['次数'].astype(str)

# 筛选保留需要的列
merged_df = merged_df[['finall', '发货金额', '地区']]

# 将数据保存为Excel文件，不保留索引
merged_df.to_excel(f'C:/Users/15516/Desktop/抽奖名单核对.xlsx', index=False)

# 重命名"finall"列为"名单"，然后拆分为每100个数据为一列
merged_df = merged_df['finall'].rename('名单')
new_df = split_df_into_columns(merged_df)

# 将拆分后的数据保存为Excel文件，不保留列名
new_df.to_excel(f'C:/Users/15516/Desktop/抽奖名单.xlsx', index=False, header=None)
